In [ ]:

import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/missionfission/dragon-project

In [31]:
# !pip install dragon-project/requirements.txt
# !pip install yamlordereddictloader
# !pip install transformers==4.0
# %cd dragon-project/src
%mkdir iters
%mkdir logs
# !ls
# !git clone https://github.com/facebookresearch/dlrm


In [13]:
%load_ext autoreload

%autoreload 2

import glob
import os
import sys
from collections import deque
from copy import deepcopy

import numpy as np
import torch
import torch.nn as nn
import yaml
import yamlordereddictloader

from torchvision import models
from yaml import dump
from scheduling import Scheduling
from generator import Generator, get_mem_props
from generator import *
from utils.visualizer import *
from ir.trace import trace
from ddfg_main import synthesis_hardware
from main import design_tech_runner, design_runner, run_single

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from common_models import alexnet_graph, vggnet_graph, resnet_graph, bert_graph, gpt2_graph, dlrm_graph, alexnet_graph, langmodel_graph
# design_runner([vggnet_graph()])
# # for node in dlrm_graph.nodes:
# #     print(node.in_edge_mem + node.mem_fetch + node.out_edge_mem, node.compute_expense )

# design_tech_runner([dlrm_graph])
vgg11_graph = vggnet_graph()
resnet_graph_data = resnet_graph()
alexnet_data = alexnet_graph()
langmod_graph1, langmod_graph2 = langmodel_graph()
# dlrm_graph_data = dlrm_graph()
bert_graph_data = bert_graph()
gpt2_graph_data = gpt2_graph()
# 
# from nonai_models.graph_processing import graph_processing

## Example 1 : Resnet 50 : Performance Estimation 

In [ ]:
run_single(resnet_graph_data, backprop=False, print_stats=True, filename = 'illusion_nvm.yaml', mapping="nn_dataflow")

## Example 2 : Non-AI Graph Processing : Architecture Synthesis and Performance Estimation 

In [ ]:
import ast

from ir.staticfg.staticfg import CFGBuilder
cfg = CFGBuilder().build_from_file(
    "aes.py",
    "nonai_models/aes.py",
)
# cfg.build_visual("aes", "pdf", show=True)
# print(cfg)

In [ ]:
synthesis_hardware("bfs")

In [ ]:
synthesis_hardware("aes")

## Architecture Generation  : Google Bert

In [ ]:
backprop = False
import time
start = time.time()
# name = ["vgg_graph", "resnet_graph","bert_graph","gpt2_graph", "dlrm_graph"]
# graph_list = [vgg11_graph, resnet_graph_data, bert_graph_data, gpt2_graph_data, dlrm_graph_data]
graph_list = [bert_graph_data]
name = ["bert_graph_data"]

for i,graph in enumerate(graph_list):
# for graph in [bert_graph_data]:
    log_file_name = "logs/"+str(name[i])+str(backprop)
    arch_time, arch_energy, arch_area = design_runner([graph], backprop, print_stats= True, stats_file = log_file_name)
# end = time.time()
# print(end-start)

## Technology Targets Generation